In [ ]:
import pandas as pd

# Amazon Books Recommender System

***Context:***

Recommender systems are widespread in all aspects of user-facing industries, from Netflix, to Amazon, to LinkedIn or to YouTube. Due to the large volumes of content, products or posts being generated daily, users would be overwhelmed with information and simple searches may not generate adequate results that may be relevant to the user.

As a result, recommender systems are a subclass of information filtering systems that provide suggestions for items that are most pertinent to a particular user. For example, over 70% of watch time on YouTube is spent watching videos the underlying recommender algorithm recommends. In this project, a recommender system will be designed for Amazon shoppers that would recommend books they would most likely enjoy based on the reviews of others.


***Problem Statement:***

_How can we accurately recommend new books to users using a user-item approach to encourage expanding user preferences to books they may not have otherwise found on their own?_


In [ ]:
# Import Ratings Data into Pandas DF
books = pd.read_csv('../data/Books.csv')
books.head()

In [ ]:
# Import Metadata about the Books into Pandas Df

import gzip 
import json

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

metadata = getDF('../data/books_meta.gz')

In [ ]:
metadata.head()

### Understanding the Chosen Algorithm


![Recommender Systems, Categorized](../imgs/recommender_types.png)


Recommender systems tend to come in two main flavors, each with their own advantages: collaborative filtering and content-based methods. And a more recent third approach that combines the best of both worlds.

This project will be taking the collaborative filtering approach. It is a technique that can filter out items that a user might like on the basis of reactions by similar users. It works by searching a large group of people and finding a smaller set of users with tastes similar to a particular user. To get recommendations, it looks at the items they like and combines them to create a ranked list of suggestions.

Descending one step further down, the collaborative filtering family has algorithms classified as either memory-based or model-based. The focus will be on a memory-based algorithm that uses item-item or item-based collaborative filtering.

In sumamry, our approach is using an item-based collaborative filtering, that falls within the memory-based category of the larger family of collaborative filtering algorithms. This approach offers several advantages including scalability, accuracy and recommendations that are much easier to exaplin to users.

### Understanding the Data

We'll be working with two datasets, one containing rows of ratings from users about a particular book (`books`) and another containing the metadata about all the books rated (`metadata`). 

The `books` dataset has rows of information in the form of `bookID userID rating timestamp`. This is the generally expected format for the data to be in for collaborative filtering based recommender algorithms. 

The `metadata` dataset has information about each book that was recommended, which include the following: 
- `asin` - ID of the product, e.g. 0000031852
- `title` - name of the product
- `feature` - bullet-point format features of the product
- `description` - description of the product
- `price` - price in US dollars (at time of crawl)
- `imageURL` - url of the product image
- `imageURLHighRes` - url of the high resolution product image
- related - related products (also bought, also viewed, bought together, buy after viewing)
- `rank` - sales rank information
- `brand` - brand name
- `category` - list of categories the product belongs to
- `tech1` - the first technical detail table of the product
- `tech2` - the second technical detail table of the product
- `similar_item` - similar product table

### Data Wrangling

In [ ]:
# Add column headings to `books` df for clarification
books1 = books.set_axis(['bookID', 'userID', 'rating', 'timestamp'], axis=1)
books1.head()

In [ ]:
# Get the shape of the `books` dataset
books1.shape

In [ ]:
# Get Descriptive Statistics
books1.describe()

## Exploratory Data Analysis

How many unique users and unique books are in the dataset?

In [ ]:
# Count number of Unique Users and Books
unique_users = books1.userID.nunique()
unique_books = books1.bookID.nunique()

In [ ]:
print(f"There are {unique_books} unique books rated by {unique_users} unique users.")
print(f"In total, there are {books1.shape[0]} total reviews.")

What is the average number of books that a user rated?

In [ ]:
avg_num_ratings = books1.groupby('userID').size().mean()

In [ ]:
print(f"The average number of books a user rated was {round(avg_num_ratings, 2)} books.")

What is the average number of ratings each book got?

In [ ]:
ratings_per_book = books1.groupby('bookID').size().mean()
print(f"The average number of ratings each book received was {round(ratings_per_book, 2)}")

What is the average price of a book bought?

In [ ]:
metadata.price.mean()